In [70]:
%run func_def.ipynb

imports loaded.

Pre-Processing func:
********************
    get_height(item)
    get_weight(item)
    get_reach(item)
    get_std_reach(item, height, reach_list, height_list)
    get_age(item)
    get_std_age(item, age_list)
    get_win_perc(name, date, df)
    get_lose_streak_data_frame(name, date, df)
    get_win_streak_data_frame(name, date, df)
    get_streak_data_frame(name, date, df)
    get_win_streak(name, date, df)
    get_lose_streak(name, date, df)
    get_fighter_stats(name, data )
    add_columns(data, column_list)

Feature Engineering func:
*************************
    get_stat(index, data, stat='height', fo='')
    get_swaped_row(index, data)
    get_wins(combo, data)
    get_wins_ratio(combo, data)

stats_model_training func:
********************************
    get_sig_features(data, sig_r)
    get_r_score(X, y)
    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)
    set_random_forest_reg(X_columns, y_columns, data)
    set_predictions(mod

### <span style='color:blue'> Insert new fight: </span> 

In [89]:
fighter = 'Islam Makhachev'
opponent = 'Alexander Volkanovski'
print(fighter + ' vs ' + opponent)

Islam Makhachev vs Alexander Volkanovski


### Fight Preview(load statistics data frame):

In [90]:
loc = '../data/'
file = 'ufc_fighters_stats_processed.csv'
stats = pd.read_csv(loc+file)
stats = stats[stats.columns[1:]]
get_fighters_stats(fighter=fighter, opponent=opponent, data=stats)

,Islam Makhachev,Alexander Volkanovski
wins,23,25
losses,1.0,1.0
draws,0,0
height,177.8,167.64
weight,69.75,65.25
reach,177.8,180.34
stance,Southpaw,Orthodox
age,31.0,34.0
slpm,2.37,6.79
str. acc.,0.59,0.57


### load main data frame:

In [91]:
path = '../data/fight_stat_data_trained.csv'
df = pd.read_csv(path)
df = df[df.columns[0:len(df.columns)-4]]
df.date = pd.to_datetime(df.date)
print('Data Frame loaded: '+str(df.shape))
df.tail(3)

Data Frame loaded: (6829, 22)


,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6826,train,2017-07-29,Kailin Curran,Aleksandra Albu,lose,U-DEC:,-0.00892,0.123128,-1.597639,0.008337,-0.150322,0.248259,-0.545754,0.199566,-0.613584,-0.320786,-0.491150,1.249440,-0.054481,-2.040434,-0.589427,2.164739
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,win,S-DEC:,-0.00892,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286
6828,validation,2017-07-29,Drew Dober,Joshua Burkman,win,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961


In [93]:
match = set_new_fight(fighter=fighter, opponent=opponent, data=df)
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,new,new,2023-02-22,new,Islam Makhachev,Alexander Volkanovski,None,None,None,None,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0,-1,0


### load trained models:

In [85]:
stat_pred_reg = load_trained_model(filename_path='../global_variables/stats_pred_random_forest_reg.pkl')
stat_pred_reg.feature_importances_
stat_pred_reg.feature_names_in_

../global_variables/stats_pred_random_forest_reg.pkl loaded.


array(['td avg._dif', 'td acc._dif', 'td def._dif', 'sub. avg._dif',
       'slpm_dif', 'str. acc._dif', 'sapm_dif', 'str. def_dif',
       'stance_win_ratio', 'win_perc_dif', 'win_strk_dif',
       'lose_strk_dif'], dtype=object)

In [96]:
stat_pred_reg.predict(match[stat_pred_reg.feature_names_in_])
stat_pred_reg.

4

In [97]:
result_pred_clf = load_trained_model(filename_path='../global_variables/result_pred_random_forest_clf.pkl')
result_pred_clf.feature_names_in_

../global_variables/result_pred_random_forest_clf.pkl loaded.


array(['kd_dif', 'str_dif', 'td_dif', 'sub_dif', 'td avg._dif',
       'td acc._dif', 'td def._dif', 'sub. avg._dif', 'slpm_dif',
       'str. acc._dif', 'sapm_dif', 'str. def_dif', 'stance_win_ratio',
       'win_perc_dif', 'win_strk_dif', 'lose_strk_dif'], dtype=object)

In [98]:
method_pred_clf = load_trained_model(filename_path='../global_variables/method_pred_random_forest_clf.pkl')
method_pred_clf.feature_names_in_

../global_variables/method_pred_random_forest_clf.pkl loaded.


array(['result', 'kd_dif', 'str_dif', 'td_dif', 'sub_dif', 'td avg._dif',
       'td acc._dif', 'td def._dif', 'sub. avg._dif', 'slpm_dif',
       'str. acc._dif', 'sapm_dif', 'str. def_dif'], dtype=object)

## <span style='color:green'> [done] </span> predict fight statistics:

In [50]:
loc = '../data/'
file = 'fight_stat_data_trained.csv'
df = pd.read_csv(loc+file)
df = df[df.columns[0:len(df.columns)-4]]
df.date = pd.to_datetime(df.date)
print('Data Frame loaded: '+str(df.shape))
df.tail(3)

Data Frame loaded: (6829, 22)


,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6826,test,2017-07-29,Kailin Curran,Aleksandra Albu,lose,U-DEC:,-0.00892,0.123128,-1.597639,0.008337,-0.150322,0.248259,-0.545754,0.199566,-0.613584,-0.320786,-0.491150,1.249440,-0.054481,-2.040434,-0.589427,2.164739
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,win,S-DEC:,-0.00892,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286
6828,train,2017-07-29,Drew Dober,Joshua Burkman,win,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961


#### <span style='color:green'> [done] </span> Add new fight to data frame:

In [51]:
df = pd.concat([df,set_new_fight(fighter=fighter, opponent=opponent, data=df)], ignore_index=True)
df.tail(3)

,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,win,S-DEC:,-0.00892,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286
6828,train,2017-07-29,Drew Dober,Joshua Burkman,win,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961
6829,new,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,new,NaN,NaN,NaN,NaN,1.710000,0.290000,0.150000,1.100000,-4.420000,0.020000,-2.580000,0.060000,-0.004158,0.000000,-1.000000,0.000000


#### <span style='color:green'> [done] </span> define X & y:

In [52]:
req_data = ['set','date','fighter','opponent','result','method']
y = ['kd_dif','str_dif','td_dif','sub_dif']
X = ['td avg._dif',
     'td acc._dif',
     'td def._dif',
     'sub. avg._dif',
     'slpm_dif',
     'str. acc._dif',
     'sapm_dif',
     'str. def_dif',
     'stance_win_ratio',
     'win_perc_dif',
     'win_strk_dif',
     'lose_strk_dif']
df = df[req_data + y + X]
df.tail(3)

,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,win,S-DEC:,-0.00892,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286
6828,train,2017-07-29,Drew Dober,Joshua Burkman,win,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961
6829,new,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,new,NaN,NaN,NaN,NaN,1.710000,0.290000,0.150000,1.100000,-4.420000,0.020000,-2.580000,0.060000,-0.004158,0.000000,-1.000000,0.000000


#### <span style='color:green'> [done] </span> get optimized regressor parameters:

In [53]:
gp = pd.read_csv('../global_variables/ml_models_parameters.csv')
mask = gp.ml_model.str.contains('stats_')
rf_reg = gp[mask].reset_index(drop=True)
opt_n_estimators = int(rf_reg.value[0])
rf_reg

,ml_model,parameter_name,value,accuracy
0,stats_random_forest_regressor,n_estimator,256,0.8852


#### <span style='color:green'> [done] </span> Train Random forest regressor model:

In [54]:
rf_reg = set_random_forest_reg(X_columns=X, y_columns=y, data=df, n_estimators=opt_n_estimators)
df = set_predictions(model=rf_reg, X_columns=X, y_columns=y, data=df)
df.tail(3)

,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,kd_dif_pred,str_dif_pred,td_dif_pred,sub_dif_pred
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,win,S-DEC:,-0.00892,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286,-0.173002,-0.173002,-0.173002,-0.173002
6828,train,2017-07-29,Drew Dober,Joshua Burkman,win,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961,0.418606,0.418606,0.418606,0.418606
6829,new,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,new,NaN,NaN,NaN,NaN,1.710000,0.290000,0.150000,1.100000,-4.420000,0.020000,-2.580000,0.060000,-0.004158,0.000000,-1.000000,0.000000,-0.098802,-0.098802,-0.098802,-0.098802


#### <span style='color:green'> [done] </span> save new fight data:

In [55]:
new_fight = df[df['set']=='new'].copy()

for stat in ['kd_dif','str_dif','td_dif','sub_dif']:
    new_fight[stat] = new_fight[stat+'_pred']
    
new_fight = new_fight[new_fight.columns[0:len(new_fight.columns)-4]].reset_index(drop=True)
new_fight

,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,new,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,new,-0.098802,-0.098802,-0.098802,-0.098802,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0.0,-1.0,0.0


In [56]:
loc = '../data/'
file = 'new_fight_data.csv'
pd.DataFrame(data=new_fight).to_csv(loc+file)
print(loc + file + ' saved.')

../data/new_fight_data.csv saved.


## <span style='color:green'> [done] </span> predict fight results:

In [57]:
loc = '../data/'
file = 'fight_result_data_trained.csv'
df = pd.read_csv(loc+file)
#df = df[df.columns[0:len(df.columns)-1]]
df.date = pd.to_datetime(df.date)
print('Data Frame loaded: '+str(df.shape))
df.tail(3)

Data Frame loaded: (6829, 23)


,set,date,fighter,opponent,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,result,result_pred
6826,test,2017-07-29,Kailin Curran,Aleksandra Albu,U-DEC:,-0.00892,0.123128,-1.597639,0.008337,-0.150322,0.248259,-0.545754,0.199566,-0.613584,-0.320786,-0.491150,1.249440,-0.054481,-2.040434,-0.589427,2.164739,lose,lose
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,S-DEC:,-0.00892,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286,win,win
6828,train,2017-07-29,Drew Dober,Joshua Burkman,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961,win,win


In [58]:
new_fight

,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,new,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,new,-0.098802,-0.098802,-0.098802,-0.098802,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0.0,-1.0,0.0


In [59]:
df = pd.concat([df,new_fight],ignore_index=True)
df.tail(3)

,set,date,fighter,opponent,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,result,result_pred
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,S-DEC:,-0.008920,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286,win,win
6828,train,2017-07-29,Drew Dober,Joshua Burkman,KO/TKO:Punch,1.344760,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961,win,win
6829,new,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,-0.098802,-0.098802,-0.098802,-0.098802,1.710000,0.290000,0.150000,1.100000,-4.420000,0.020000,-2.580000,0.060000,-0.004158,0.000000,-1.000000,0.000000,new,NaN


#### <span style='color:green'> [done] </span> get optimized Random Forest classifier parameters:

In [60]:
gp = pd.read_csv('../global_variables/ml_models_parameters.csv')
mask = gp.ml_model.str.contains('result_')
rf_clf = gp[mask].reset_index(drop=True)
opt_n_estimators = int(rf_clf.value[0])
rf_clf

,ml_model,parameter_name,value,accuracy
0,result_random_forest_classifier,n_estimator,32,0.85792


#### <span style='color:green'> [done] </span> Train classifier with optimized parameters:

In [61]:
req_data = ['set','date','fighter','opponent','method']
y = ['result']
X = ['kd_dif','str_dif','td_dif','sub_dif',
    'td avg._dif','td acc._dif','td def._dif','sub. avg._dif',
    'slpm_dif','str. acc._dif','sapm_dif','str. def_dif',
    'stance_win_ratio','win_perc_dif','win_strk_dif','lose_strk_dif']

df = df[req_data + X + y]
df.tail(3)

,set,date,fighter,opponent,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,result
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,S-DEC:,-0.008920,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286,win
6828,train,2017-07-29,Drew Dober,Joshua Burkman,KO/TKO:Punch,1.344760,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961,win
6829,new,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,-0.098802,-0.098802,-0.098802,-0.098802,1.710000,0.290000,0.150000,1.100000,-4.420000,0.020000,-2.580000,0.060000,-0.004158,0.000000,-1.000000,0.000000,new


In [64]:
set_predictions(model=rf_clf, data=df, X_columns=X, y_columns=y)

,set,date,fighter,opponent,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,result,result_pred
0,train,2022-09-03,Tai Tuivasa,Ciryl Gane,KO/TKO:Punches,-0.008920,-3.061301,-0.018213,0.008337,-0.344915,-0.735658,-0.107919,-0.470979,-0.657553,-0.808609,1.358824,-1.448854,1.671822,-0.378408,2.840653,-1.062286,lose,lose
1,train,2022-09-03,Robert Whittaker,Marvin Vettori,U-DEC:,-0.008920,1.509291,0.376643,0.008337,-0.595105,-0.243700,0.329916,-0.566771,0.152724,-0.158179,-0.190989,0.153258,-1.281305,0.132985,-0.589427,1.089064,win,win
2,train,2022-09-03,Joaquin Buckley,Nassourdine Imavov,U-DEC:,-0.008920,-0.214047,-0.807926,-1.691263,0.305581,0.283399,-0.208958,-1.620484,-0.305805,-1.540343,-0.166486,-0.352672,1.671822,-0.097141,0.553933,0.013389,lose,lose
3,train,2022-09-03,Alessio Di Chirico,Roman Kopylov,KO/TKO:Punches,-1.362601,-0.776005,-0.018213,0.008337,0.466815,-0.384259,-0.208958,0.007982,-0.789458,-0.889913,-0.313504,0.068937,-1.281305,1.027923,-0.017747,-1.062286,lose,lose
4,train,2022-09-03,Jarno Errens,William Gomis,U-DEC:,-0.008920,-0.476294,-1.202783,0.858138,-1.668145,-2.106113,-2.027658,0.965903,-0.525647,-0.646001,0.495094,-0.689958,-1.281305,0.005137,-0.017747,0.013389,lose,lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6825,train,2017-07-29,Calvin Kattar,Andre Fili,U-DEC:,-0.008920,0.048200,0.771499,0.008337,-1.089927,-0.700518,0.734072,-0.183603,0.787126,0.167036,1.885637,0.153258,-0.054481,-1.452333,-0.589427,0.013389,win,win
6826,train,2017-07-29,Kailin Curran,Aleksandra Albu,U-DEC:,-0.008920,0.123128,-1.597639,0.008337,-0.150322,0.248259,-0.545754,0.199566,-0.613584,-0.320786,-0.491150,1.249440,-0.054481,-2.040434,-0.589427,2.164739,lose,lose
6827,train,2017-07-29,Jarred Brooks,Eric Shelton,S-DEC:,-0.008920,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286,win,win
6828,train,2017-07-29,Drew Dober,Joshua Burkman,KO/TKO:Punch,1.344760,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961,win,win


In [62]:
k = 10
size = len(df)-1
batch = int(size/k)
acc = []
k_predictions = []

for i in tqdm(range(0,size-batch,batch)):
    test_start = i
    test_end = i + batch - 1
    df.loc[:,'set'] = 'train'
    df.loc[test_start:test_end,'set'] = 'test'
    rf_clf = set_random_forest_clf(X_columns=X, y_columns=y, data=df, n=opt_n_estimators)
    df = set_predictions(model=rf_clf, data=df, X_columns=X, y_columns=y) 
    y_test_act = df['result'][df.set=='test']
    y_test_pred = df['result_pred'][df.set=='test']
    acc.append(get_accuracy(act=y_test_act, pred=y_test_pred))
    k_predictions.append(df.tail(1).result_pred) #get specific new fight results

opt_acc = np.mean(acc)
final_pred = np.max(k_predictions)

print('\n%d-fold Cross-Validation model:'%k)
print('------------------------------')
print('prediction: %s' %final_pred)
print('accuracy: %.4f' %opt_acc)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.48it/s]


10-fold Cross-Validation model:
------------------------------
prediction: new
accuracy: 0.8689


In [63]:
df.loc[len(df)-1,'result'] = final_pred
df.loc[len(df)-1,'result_pred'] = final_pred
df.tail(1)

,set,date,fighter,opponent,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,result,result_pred
6829,train,2023-02-16,Islam Makhachev,Alexander Volkanovski,new,-0.098802,-0.098802,-0.098802,-0.098802,1.71,0.29,0.15,1.1,-4.42,0.02,-2.58,0.06,-0.004158,0.0,-1.0,0.0,new,new


In [29]:
new_fight = df.tail(1)[df.columns[0:len(df.columns)]]
new_fight

,set,date,fighter,opponent,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif,result
6828,train,2017-07-29,Drew Dober,Joshua Burkman,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.01209,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961,win


In [ ]:
df = pd.concat([df,new_fight],ignore_index=True)
df.tail(3)

In [ ]:
>>>>>>>>>>>>>>

#### <span style='color:green'> [done] </span> Final Prediction:

In [19]:
param = pd.read_csv('../global_variables/ml_models_parameters.csv').drop(columns=['Unnamed: 0'])
param

KeyError: "['Unnamed: 0'] not found in axis"

In [65]:
print('--------------------------------------')
print('%s vs %s:' %(fighter, opponent))
print('--------------------------------------')
print('Winner: ' + get_winner(fighter=fighter, opponent=opponent, data=df))
print('Pridiction Accuracy: %.5f' %float(param.accuracy[param.ml_model=='knn_clf']))
print('--------------------------------------')
get_fighters_stats(fighter=fighter, opponent=opponent, data=stats)

--------------------------------------
Islam Makhachev vs Alexander Volkanovski:
--------------------------------------
Winner: Alexander Volkanovski


NameError: name 'param' is not defined